# Introducción a la Inteligencia Artificial

## CLASE 3

## Referencias

- [Cheatsheet de Latex](https://www.caam.rice.edu/~heinken/latex/symbols.pdf)
- [DRIVE](https://drive.google.com/drive/folders/1TDRvlwfSgLI39gHXgVhWlSaRJ-aTSaDE?usp=sharing)

## Comentarios sobre el estado del documento



## Importar librerías

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split

# MATPLOTLIB & SNS
# from mpl_toolkits import mplot3d
# from matplotlib import cm
# from matplotlib_venn import venn2
# import seaborn as sns

# MATH & RANDOM
# import math
# import numpy.random as random
# import scipy.stats as stats
# random.seed(42)

#PANDAS
# import pandas as pd

# %matplotlib inline

In [49]:
# Ejercicio donde vamos a computar el gradiente descendente

#1) plantear W
#2) calcualr error
#3) caluclar función costo
#4) calcular el gradiente de la función costo para ese W
#5) mejorar el W restando alpha por el gradiente

FILE_PATH = "C:/Users/jota_/00_Especialización_IA/00_Recursos/01_DataSets/income.data.csv"


class Data(object):

    def __init__(self, path):
        self.dataset = self._build_dataset(path)

    def split(self, percentage):
        # divides the dataset by permutating, masking and slicing into train and split
        SIZE = self.dataset.shape[0]
        data_total = self.dataset
        idx = np.arange(0, SIZE)
        
        # permutate
        perm_idx = np.random.permutation(idx)
        
        # generate index slice
        train_idx = perm_idx[:int(SIZE * percentage)]
        test_idx = perm_idx[int(SIZE * percentage):]
        
        # slice dataset
        train_data = data_total[train_idx]
        test_data = data_total[test_idx]
        
        def vector_creator(vector, dataset):
            x = dataset[vector].reshape(dataset[vector].shape[0], 1)
            return x
        
        X_train = vector_creator('X', train_data)
        y_train = vector_creator('y', train_data)
        X_test = vector_creator('X', test_data)
        y_test = vector_creator('y', test_data)

        return X_train, y_train, X_test, y_test

    def _build_dataset(self, path):
        ## creates structured array
        structure = [('X', np.float32), ('y', np.float32)]
        structure = np.dtype(structure)
        # load numpy array from disk using a generator
        with open(path, encoding="utf8") as file:
                    data_gen = ((line.split(',')[1], line.split(',')[2]) 
                                for line in file if not line.split(',')[1][2].isalpha())
                    data_total = np.fromiter(data_gen, structure)
        
        return data_total




class BaseModel(object):
    
    def __init__(self):
        self.predict
        

class Linear_regression_gradient_descent(BaseModel):
    
    def __init__(self):
        
        return None
    
    def fit(self, X, y, alpha = 0.01, epochs=100 ):
        
        X = np.hstack((X_train, np.ones([X_train.shape[0], 1])))
        
        n = X.shape[0]
        m = X.shape[1]
        
        W = np.random.randint(-100, 100, size=[2,1])
        
        
        for i in range(epochs):
            prediction = np.matmul(X, W)
            
            error = y - prediction
            
            
            grad_sum = np.sum(error * X, axis=0)
            grad_mul = -2/n * grad_sum
            gradient = grad_mul.T
            print(gradient)
            W = W - (alpha * gradient)
            
        return W
    
    
class Linear_regression_gradient_stochastic(BaseModel):
    
    def __init__(self):
        
        return None
    
    def fit(self, X, y, alpha = 0.01, epochs=100 ):
        
        X = np.hstack((X_train, np.ones([X_train.shape[0], 1])))
        
        n = X.shape[0]
        m = X.shape[1]
        
        W = np.random.randint(-100, 100, size=[2,1])
        
        
        for i in range(epochs):
            prediction = np.matmul(X, W)
            
            error = y - prediction
            
            
            grad_sum = np.sum(error * X, axis=0)
            grad_mul = -2/n * grad_sum
            gradient = grad_mul.T
            W = W - (alpha * gradient)
            
        return W
    

In [50]:

income_happi = Data(FILE_PATH)
X_train, y_train, X_test, y_test = income_happi.split(0.8)

model = Linear_regression_gradient_descent()

model.fit(X_train, y_train)

[-593.54226779 -111.80513778]
[-270.46781705  -99.65040764]
[-123.24790336  -88.81706101]
[-56.16211884 -79.16144562]
[-25.59218864 -70.55552618]
[-11.6619553  -62.88518654]
[ -5.31416845 -56.04871653]
[ -2.42158245 -49.95546323]
[ -1.1034768  -44.52462896]
[ -0.50283691 -39.6841998 ]
[ -0.22913482 -35.36999073]
[ -0.10441311 -31.52479452]
[ -0.0475794  -28.09762312]
[-2.16811805e-02 -2.50430316e+01]
[-9.87977080e-03 -2.23205155e+01]
[-4.50205518e-03 -1.98939737e+01]
[-2.05151529e-03 -1.77312297e+01]
[-9.34843047e-04 -1.58036053e+01]
[-4.25993180e-04 -1.40855398e+01]
[-1.94118350e-04 -1.25542511e+01]
[-8.84566595e-05 -1.11894343e+01]
[-4.03082997e-05 -9.97299151e+00]
[-1.83678542e-05 -8.88879249e+00]
[-8.36994044e-06 -7.92246057e+00]
[-3.81404937e-06 -7.06118199e+00]
[-1.73800192e-06 -6.29353604e+00]
[-7.91979954e-07 -5.60934358e+00]
[-3.60892726e-07 -4.99953209e+00]
[-1.64453088e-07 -4.45601535e+00]
[-7.49386875e-08 -3.97158624e+00]
[-3.41483469e-08 -3.53982112e+00]
[-1.55608685e-08 -

array([[-4.09562725, -4.71572121],
       [24.90437275, 24.28427879]])

In [32]:
income_happi = Data(FILE_PATH)
X_train, y_train, X_test, y_test = income_happi.split(0.8)

np.hstack((X_train, np.ones([X_train.shape[0], 1]))).shape

(398, 2)

In [25]:
np.ones([X_train.shape[0], 1]).shape

(398, 1)